## Simple Mission in Auto-Mode

This notebook serves to check the ardupilot installation and debugging

In [7]:
import os

from simulator.config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from simulator.helpers.cleanup import clean
from simulator.helpers.connections import create_tcp_conn, wait_for_port
from simulator.helpers.coordinates import GRAPose
from simulator.helpers.processes import create_process
from simulator.helpers.setup_log import setup_logging
from simulator.planner import AutoPlan, Plan, State

clean()

## Launch Copter (ardupilot)

In [8]:
gra_origin = GRAPose(lat=-35.3633245,lon=149.1652241,alt=0,heading=0)
spawn_str = gra_origin.to_str()



sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I0 --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )
wait_for_port(port = BasePort.ARP, timeout = 0.5)



Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...


## 1. Connect to the vehicle

In [9]:
conn = create_tcp_conn(
    base_port=BasePort.ARP, 
    offset=0,
    role="client", 
    src_sysid=sysid,
    src_compid=140
)

print("✅ TCP connection established!")

✅ TCP connection established!


## Create auto plan

In [10]:
mission_path = "simulator/planner/missions/square_mission.waypoints"
plan = AutoPlan(name="basic_auto_plan", mission_path=mission_path)
plan

🕓 <AutoPlan '📋 basic_auto_plan'>
  🕓 <Action '💾 UPLOAD_MISSION'>
    🕓 <ClearMission '🔹 clear previous mission'>
    🕓 <UploadMission '🔹 upload mission'>
  🕓 <Action '🛡️ PREARM'>
    🕓 <CheckDisarmed '🔹 Check disarmed'>
    🕓 <EKFStatus '🔹 Check EKF status'>
    🕓 <GPSStatus '🔹 Check GPS'>
    🕓 <CheckSystem '🔹 Check system status'>
  🕓 <Action '⚙️ MODE'>
    🕓 <SwitchMode '🔹 Switch to GUIDED'>
  🕓 <Action '🔒 ARM'>
    🕓 <Arm '🔹 arm'>
  🕓 <Action '🚀 START_MISSION'>
    🕓 <StartMission '🔹 start mission'>
  🕓 <Action '👀 MONITOR_MISSION'>
    🕓 <CheckItems '🔹 check items'>
    🕓 <CheckEndMission '🔹 check end mission'>

# Save mission

In [11]:
enu_path = Plan.create_square_path(side_len=5, alt=5, clockwise=True)
gra_path=GRAPose.unpose_all(gra_origin.to_abs_all(enu_path))
plan.save_basic_mission(sysid=sysid,gra_wps=gra_path)

# Execute Auto Plan

In [12]:
setup_logging('plan',verbose=1)
plan.bind(conn,gra_origin.unpose())
while plan.state != State.DONE:
    plan.act()

23:58:06 - plan - INFO - 🧹 Vehicle 1: Cleared previous mission
23:58:06 - plan - INFO - ✅ Vehicle 1: 7 waypoints read
23:58:07 - plan - INFO - ✅ Vehicle 1: Mission upload successful!
23:58:07 - plan - INFO - ✅ Vehicle 1: Action Done: 💾 UPLOAD_MISSION
23:58:48 - plan - INFO - ✅ Vehicle 1: Action Done: 🛡️ PREARM
23:58:48 - plan - INFO - ✅ Vehicle 1: Action Done: ⚙️ MODE
23:58:49 - plan - INFO - ✅ Vehicle 1: Action Done: 🔒 ARM
23:58:49 - plan - INFO - 🚀 Vehicle 1: Mission has started
23:58:49 - plan - INFO - ✅ Vehicle 1: Action Done: 🚀 START_MISSION
23:58:49 - plan - INFO - 📦 Vehicle 1 has 7 mission items
23:58:49 - plan - INFO - Vehicle 1: ⭐ Reached item: 0
23:58:49 - plan - INFO - Vehicle 1: 📍 Target Position: ENU(x=-0.18, y=-0.28, z=5.0)
23:58:57 - plan - INFO - Vehicle 1: ⭐ Reached item: 1
23:58:57 - plan - INFO - Vehicle 1: 📍 Target Position: ENU(x=-0.18, y=5.22, z=5.0)
23:59:03 - plan - INFO - Vehicle 1: ⭐ Reached item: 2
23:59:03 - plan - INFO - Vehicle 1: 📍 Target Position: ENU(x=

In [13]:
plan

✅ <AutoPlan '📋 basic_auto_plan'>
  ✅ <Action '💾 UPLOAD_MISSION'>
    ✅ <ClearMission '🔹 clear previous mission'>
    ✅ <UploadMission '🔹 upload mission'>
  ✅ <Action '🛡️ PREARM'>
    ✅ <CheckDisarmed '🔹 Check disarmed'>
    ✅ <EKFStatus '🔹 Check EKF status'>
    ✅ <GPSStatus '🔹 Check GPS'>
    ✅ <CheckSystem '🔹 Check system status'>
  ✅ <Action '⚙️ MODE'>
    ✅ <SwitchMode '🔹 Switch to GUIDED'>
  ✅ <Action '🔒 ARM'>
    ✅ <Arm '🔹 arm'>
  ✅ <Action '🚀 START_MISSION'>
    ✅ <StartMission '🔹 start mission'>
  ✅ <Action '👀 MONITOR_MISSION'>
    ✅ <CheckItems '🔹 check items'>
    ✅ <CheckEndMission '🔹 check end mission'>